Purpose of this Notebook
1. Test ACIS API
2. Retrieve list of stations in AK with recent valid data 
3. Write out stations to CSV and GeoPackage (for use in GIS)


In [ ]:
import requests
import pandas as pd
import geopandas as gp
import numpy as np
from pathlib import Path

### Helpers

In [ ]:
PROJPATH = Path().resolve().parent
outdir = PROJPATH / 'data/working/'

In [ ]:
def safelyget(alist, idx, default='N/A'):
    """Returns alist[idx] if exists, else default"""
    try:
        return alist[idx]
    except (KeyError, IndexError):
        return default
    
def pick_icao(sidslist):
    """Returns the ICAO code from a station ID list from ACIS"""
    try:
        return [item.split()[0] for item in sidslist if item.endswith(' 5')][0]
    except (AttributeError, IndexError):
        # There's no ICAO code in the list
        return ''

### Get all ACIS stations that have avg daily temp data, with their data validity rage, ID and lat/lon

Variable 13 = snow water equivalent

In [ ]:
baseurl = 'http://data.rcc-acis.org/StnMeta'
params = {
    'state': "AK",
    'meta': "name,uid,sids,ll,elev,valid_daterange",
    'elems': "avgt,snow"
}

In [ ]:
resp = requests.get(url=baseurl, params=params)

In [ ]:
stationdat = resp.json()['meta']
len(stationdat)

1115

In [ ]:
stationdat

[{'name': 'MCKINLEY NATIONAL PARK AP',
  'll': [-148.91667, 63.73333],
  'sids': ['46403 1',
   '26489 1',
   '506089 2',
   'INR 3',
   '5MK 3',
   'PAIN 5',
   'USC00506089 6',
   'INR 7',
   '5MK 7',
   'USW00046403 32'],
  'elev': 1720.0,
  'valid_daterange': [['1949-09-01', '1950-02-28'],
   ['1949-09-01', '1950-02-28']],
  'uid': 1},
 {'name': 'BIG RIVER LAKES',
  'll': [-152.29694, 60.81417],
  'sids': ['26557 1',
   '500788 2',
   'LVR 3',
   '5BI 3',
   'PALV 5',
   'USC00500788 6',
   '5BI 7',
   'USW00026557 32'],
  'elev': 60.0,
  'valid_daterange': [['1978-01-01', '2010-11-30'],
   ['1978-01-01', '2010-11-30']],
  'uid': 450},
 {'valid_daterange': [['1949-01-01', '1965-10-29'],
   ['1949-01-01', '1950-11-20']],
  'll': [179.25, 51.38333],
  'sids': ['45702 1', 'USW00045702 6'],
  'name': 'AMCHITKA ISLAND',
  'uid': 20416},
 {'name': 'AMCHITKA',
  'll': [179.28333, 51.38333],
  'sids': ['500252 2', 'AHT 3', 'USC00500252 6', 'AHT 7'],
  'elev': 223.0,
  'valid_daterange': [[

In [ ]:
records = [   
    dict(
        name=stat['name'],
        acisID=stat['uid'],
        icao=pick_icao(stat['sids']),
        longitude=safelyget(safelyget(stat, 'll', []), 0, np.nan),
        latitude=safelyget(safelyget(stat, 'll', []), 1, np.nan),
        elev_ft=safelyget(stat, 'elev', np.nan), 
        valid_avgT_start=safelyget(stat['valid_daterange'][0], 0, ''),
        valid_avgT_end=safelyget(stat['valid_daterange'][0], 1, ''),
        valid_snowdepth_start=safelyget(stat['valid_daterange'][1], 0, ''),
        valid_snowdepth_end=safelyget(stat['valid_daterange'][1], 1, ''),
    )
    for stat in stationdat
]


In [ ]:
stationDF = pd.DataFrame.from_records(records)
stationDF

,name,acisID,icao,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end
0,MCKINLEY NATIONAL PARK AP,1,PAIN,-148.91667,63.73333,1720.0,1949-09-01,1950-02-28,1949-09-01,1950-02-28
1,BIG RIVER LAKES,450,PALV,-152.29694,60.81417,60.0,1978-01-01,2010-11-30,1978-01-01,2010-11-30
2,AMCHITKA ISLAND,20416,,179.25000,51.38333,NaN,1949-01-01,1965-10-29,1949-01-01,1950-11-20
3,AMCHITKA,20418,,179.28333,51.38333,223.0,1943-02-01,1993-08-31,1943-11-01,1993-08-31
4,ASI TANAGA ISLAND NS,20425,,-177.94444,51.76111,NaN,1948-09-04,1949-03-16,1948-09-04,1949-03-16
...,...,...,...,...,...,...,...,...,...,...
1110,JUNEAU 12.2 SW,113241,,-134.39751,58.27964,37.0,,,2023-08-21,2023-08-30
1111,HUSLIA 27 E,114109,,-155.47610,65.65820,207.0,2024-12-06,2025-03-17,,
1112,FOX 3.8 SW,114989,,-147.70297,64.91725,1245.0,,,2023-11-09,2025-03-14
1113,PALMER 2.6 NW,115060,,-149.16159,61.62753,457.0,,,2024-08-21,2025-03-17


Let's drop stations with no valid temperature data as well as too short / old temp series

In [ ]:
stationDF = stationDF[~(stationDF.valid_avgT_start=='')] 
stationDF = stationDF[stationDF.valid_avgT_end > '1990']
stationDF = stationDF.sort_values(by='name')
stationDF = stationDF.reset_index(drop=True)
stationDF

,name,acisID,icao,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end
0,ADAK AP,20427,PADK,-176.64999,51.88333,17.0,1942-10-01,2025-03-16,1942-11-01,1996-03-31
1,AKHIOK,106217,PAKH,-154.18333,56.93333,43.0,2022-10-20,2025-01-28,,
2,AKUTAN,21171,,-165.76666,54.13333,NaN,1986-01-01,1990-02-22,1986-01-01,1990-02-22
3,ALASKA PACIFIC UNIVERSITY,21216,,-149.80556,61.18889,220.0,1993-07-01,2004-10-31,1993-07-01,2004-10-31
4,ALCAN HWY MI-1244 ALASKA,65647,,-141.46671,62.81670,1800.0,1990-06-18,2023-12-02,,
...,...,...,...,...,...,...,...,...,...,...
634,YAKUTAT AIRPORT,20602,PAYA,-139.67104,59.51211,41.0,1917-04-01,2025-03-16,1917-04-01,2024-07-06
635,YAKUTAT POWER,94427,,-139.85556,59.72861,71.0,2019-12-01,2020-03-01,2019-12-01,2020-02-12
636,YANTARNI BAY ALASKA,83027,,-157.03329,56.91440,96.0,2010-08-26,2025-03-16,,
637,Yakutat Area,32886,,NaN,NaN,NaN,1917-05-01,2025-03-16,1917-08-01,2024-07-06


In [ ]:
stationDF.to_csv(outdir / "ACIS_stations_AK_cleaned.csv")

In [ ]:
selectedstationsDF = stationDF[
    (stationDF.valid_avgT_start < '1980-01-01') & (stationDF.valid_avgT_end > '2025-02-01')]
selectedstationsDF

,name,acisID,icao,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end,geometry
0,ADAK AP,20427,PADK,-176.64999,51.88333,17.0,1942-10-01,2025-03-16,1942-11-01,1996-03-31,POINT (-176.64999 51.88333)
9,ALYESKA,20684,,-149.11079,60.95830,272.0,1963-11-07,2025-03-17,1963-11-07,2025-03-17,POINT (-149.11079 60.95830)
27,ANCHORAGE MERRILL FIELD,20717,PAMR,-149.85724,61.21630,126.0,1914-11-01,2025-03-16,1916-02-16,2005-02-20,POINT (-149.85724 61.21630)
30,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT,20705,PANC,-150.02771,61.16916,125.0,1952-04-01,2025-03-16,1953-02-01,2025-03-16,POINT (-150.02771 61.16916)
35,ANIAK AIRPORT,20756,PANI,-159.54305,61.58159,85.0,1920-11-08,2025-03-16,1920-08-01,1990-03-31,POINT (-159.54305 61.58159)
...,...,...,...,...,...,...,...,...,...,...,...
621,WHITE MOUNTAIN,20942,PAWM,-163.39999,64.68333,20.0,1922-01-05,2025-03-16,1922-01-14,1990-05-31,POINT (-163.39999 64.68333)
629,WISEMAN,21043,,-150.10957,67.40822,1181.0,1919-11-01,2025-03-17,1918-11-01,2025-02-25,POINT (-150.10957 67.40822)
632,WRANGELL AP,29707,PAWG,-132.38741,56.47320,56.0,1917-11-02,2025-03-16,1917-12-05,2013-02-19,POINT (-132.38741 56.47320)
634,YAKUTAT AIRPORT,20602,PAYA,-139.67104,59.51211,41.0,1917-04-01,2025-03-16,1917-04-01,2024-07-06,POINT (-139.67104 59.51211)


In [ ]:
selectedstationsDF.to_csv(outdir / "ACIS_stations_AK_recentTemps.csv")

In [ ]:
stationsGDF = gp.GeoDataFrame(
    selectedstationsDF, geometry=gp.points_from_xy(selectedstationsDF.longitude, selectedstationsDF.latitude))
stationsGDF = stationsGDF.set_crs('epsg:4326')
stationsGDF

,name,acisID,icao,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end,geometry
0,ADAK AP,20427,PADK,-176.64999,51.88333,17.0,1942-10-01,2025-03-16,1942-11-01,1996-03-31,POINT (-176.64999 51.88333)
9,ALYESKA,20684,,-149.11079,60.95830,272.0,1963-11-07,2025-03-17,1963-11-07,2025-03-17,POINT (-149.11079 60.95830)
27,ANCHORAGE MERRILL FIELD,20717,PAMR,-149.85724,61.21630,126.0,1914-11-01,2025-03-16,1916-02-16,2005-02-20,POINT (-149.85724 61.21630)
30,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT,20705,PANC,-150.02771,61.16916,125.0,1952-04-01,2025-03-16,1953-02-01,2025-03-16,POINT (-150.02771 61.16916)
35,ANIAK AIRPORT,20756,PANI,-159.54305,61.58159,85.0,1920-11-08,2025-03-16,1920-08-01,1990-03-31,POINT (-159.54305 61.58159)
...,...,...,...,...,...,...,...,...,...,...,...
621,WHITE MOUNTAIN,20942,PAWM,-163.39999,64.68333,20.0,1922-01-05,2025-03-16,1922-01-14,1990-05-31,POINT (-163.39999 64.68333)
629,WISEMAN,21043,,-150.10957,67.40822,1181.0,1919-11-01,2025-03-17,1918-11-01,2025-02-25,POINT (-150.10957 67.40822)
632,WRANGELL AP,29707,PAWG,-132.38741,56.47320,56.0,1917-11-02,2025-03-16,1917-12-05,2013-02-19,POINT (-132.38741 56.47320)
634,YAKUTAT AIRPORT,20602,PAYA,-139.67104,59.51211,41.0,1917-04-01,2025-03-16,1917-04-01,2024-07-06,POINT (-139.67104 59.51211)


In [ ]:
stationsGDF

,name,acisID,icao,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end,geometry
0,ADAK AP,20427,PADK,-176.64999,51.88333,17.0,1942-10-01,2025-03-16,1942-11-01,1996-03-31,POINT (-176.64999 51.88333)
9,ALYESKA,20684,,-149.11079,60.95830,272.0,1963-11-07,2025-03-17,1963-11-07,2025-03-17,POINT (-149.11079 60.95830)
27,ANCHORAGE MERRILL FIELD,20717,PAMR,-149.85724,61.21630,126.0,1914-11-01,2025-03-16,1916-02-16,2005-02-20,POINT (-149.85724 61.21630)
30,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT,20705,PANC,-150.02771,61.16916,125.0,1952-04-01,2025-03-16,1953-02-01,2025-03-16,POINT (-150.02771 61.16916)
35,ANIAK AIRPORT,20756,PANI,-159.54305,61.58159,85.0,1920-11-08,2025-03-16,1920-08-01,1990-03-31,POINT (-159.54305 61.58159)
...,...,...,...,...,...,...,...,...,...,...,...
621,WHITE MOUNTAIN,20942,PAWM,-163.39999,64.68333,20.0,1922-01-05,2025-03-16,1922-01-14,1990-05-31,POINT (-163.39999 64.68333)
629,WISEMAN,21043,,-150.10957,67.40822,1181.0,1919-11-01,2025-03-17,1918-11-01,2025-02-25,POINT (-150.10957 67.40822)
632,WRANGELL AP,29707,PAWG,-132.38741,56.47320,56.0,1917-11-02,2025-03-16,1917-12-05,2013-02-19,POINT (-132.38741 56.47320)
634,YAKUTAT AIRPORT,20602,PAYA,-139.67104,59.51211,41.0,1917-04-01,2025-03-16,1917-04-01,2024-07-06,POINT (-139.67104 59.51211)


In [ ]:
stationsGDF.to_file(outdir / "ACISAK_selectedstations.gpkg", driver='GPKG')